- [go doc effective_go](https://go.dev/doc/effective_go)
- jupyter kernel: [GoNB](https://github.com/janpfeifer/gonb)

## 0. fmt

[fmt doc](https://pkg.go.dev/fmt)

General:

| verb | doc |
|:--|:--|
| `%v`| the value in a default format, when printing structs, the plus flag (%+v) adds field names |
| `%#v`| a Go-syntax representation of the value |
| `%T`| a Go-syntax representation of the type of the value |
| `%%`| a literal percent sign; consumes no value |

Integer:

| verb | doc |
|:--|:--|
| `%b`| base 2 |
| `%c`| the character represented by the corresponding Unicode code point |
| `%d`| base 10 |

- `any`: go 1.18 alias of interface{}: `type any = interface{}`
- `Type assertions`: For an expression x of interface type, but not a type parameter, and a type T, the primary expression:  
  `x.(T)`

## 1. variable

`_`: blank identifier, 用来存储不需要的导包和变量

In [33]:
%main
// 变量
var _ int
_ = 10

var _ = 10

var (
    _ int
    _ = 'a'
)
var _, _ = 1, 1
v1, v2 := 1, 1
println(v1, v2)


// 常量
const _ = 10

// 常量中 iota: 整数序号
const (
    c31 = iota
    c32, c33 = iota, iota + 1
    c34, c35
    c36 = 100 * iota
)
println(c31, c32, c33, c34, c35, c36)

1 1
0 1 2 2 3 300


## 2. function

In [37]:
/*
func arrayStat(arr []int) (int, int) {
    sum, avg := 0, 0
    for idx, value := range arr {
        sum += value
        fmt.Printf("idx: %d, value: %d\n", idx, value)
    }
    return sum, avg
}
*/

// Named result parameters
func arrayStat(arr []int) (sum int, avg int) {
    for idx, value := range arr {
        sum += value
        fmt.Printf("idx: %d, value: %d\n", idx, value)
    }
    return sum, avg
}

func main() {
    fmt.Println("main")

    arr := []int{1, 2, 3}
    sum, _ := arrayStat(arr)
    fmt.Println("\nsum:", sum, "\n")
}

main
idx: 0, value: 1
idx: 1, value: 2
idx: 2, value: 3

sum: 6 



### defer

The arguments to the deferred function (which include the receiver if the function is a method) are evaluated when the defer executes, not when the call executes.

defer 的参数是在使用关键字 defer 处确定的，而不是真正执行时，所以不要担心 defer 的参数被改变

In [39]:
func withDefer(a int) int {
    fmt.Println("withDefer enter...")
    defer fmt.Println("defer in withDefer")
    return a
}

func main() {
    fmt.Println("main")

    a := withDefer(2)
    fmt.Println("withDefer result:", a)
    

    for i := 0; i < 5; i++ {
        defer fmt.Printf(" >>%d", i)
    }
}

main
withDefer enter...
defer in withDefer
withDefer result: 2
 >>4 >>3 >>2 >>1 >>0

## 3. pass by pointers vs. values

In [30]:
func passByValue(a int) int {
    fmt.Printf("org: %d", a)
    a += 10
    fmt.Printf(", after: %d\n", a)
    return a
}

func passValuePoint(pa *int) int {
    fmt.Printf("org: %d", *pa)
    *pa += 20
    fmt.Printf(", after: %d\n", *pa)
    return *pa
}

func swap(pa *int, pb *int) {
    var temp int
    temp = *pa
    *pa = *pb
    *pb = temp
}

func main() {
    var a int
    fmt.Printf("a: %d\n", a)
    b := passByValue(a);
    fmt.Printf("a: %d, b: %d\n", a, b)
    
    fmt.Printf("\npoint:\na: %d\n", a)
    c := passValuePoint(&a)
    fmt.Printf("a: %d, c: %d\n", a, c)
    
    fmt.Printf("\nswap by point:\nb: %d, c: %d\n", b, c)
    swap(&b, &c)
    fmt.Printf("b: %d, c: %d\n", b, c)
}

a: 0
org: 0, after: 10
a: 0, b: 10

point:
a: 0
org: 0, after: 20
a: 20, c: 20

swap by point:
b: 10, c: 20
b: 20, c: 10


## 4. arrays and slice

### 4.1 arrays

- Arrays are **values**. Assigning one array to another copies all the elements.
- In particular, if you pass an array to a function, it will receive a copy of the array, not a pointer to it.
- The size of an array is part of its type. The types [10]int and [20]int are distinct.

In [2]:
func prtA3(arr [3]int) {
    // if reflect.TypeOf(v).Kind() == reflect.Slice || reflect.TypeOf(v).Kind() == reflect.Array {
        // arr := v.([3]int) // cannot cast to []any
        fmt.Printf("array: %v,\t type: %T,\t len: %d,\t cap: %d\n", arr, arr, len(arr), cap(arr))
    // }
}

In [36]:
// array 是值类型，赋值和传递都是数组的复制; 数组长度也是类型的一部分, [3]int 与 [2]int 不一样
func modifyArray(arr [3]int) {
    // a copy of the org array
    arr[0] = 100
    prtA3(arr)

    arr2 := arr
    arr2[0] = 200
    prtA3(arr2)
}

func iterArr(arr [3]int) {
    for idx, value := range arr {
        fmt.Printf("%d=%d; ", idx, value)
    }
    fmt.Println("")
}

func main() {
    var arr [3]int
    arr = [3]int{1}
    prtA3(arr)
    // part of array
    fmt.Printf("arr[1:2]: %+v\n", arr[1:2])
    iterArr(arr)
    modifyArray(arr)
    prtA3(arr)
    arr2 := [3]int{1, 2}
    prtA3(arr2)
}

array: [1 0 0],	 type: [3]int,	 len: 3,	 cap: 3
arr[1:2]: [0]
0=1; 1=0; 2=0; 
array: [100 0 0],	 type: [3]int,	 len: 3,	 cap: 3
array: [200 0 0],	 type: [3]int,	 len: 3,	 cap: 3
array: [1 0 0],	 type: [3]int,	 len: 3,	 cap: 3
array: [1 2 0],	 type: [3]int,	 len: 3,	 cap: 3


### 4.2 slice

In [61]:
// go 1.18+ generics
func prtSlice[T int | float32 | string](varName string, slice []T) {
    if slice != nil {
        fmt.Printf("%s:\t %v, type: %T,\t len: %d,\t cap: %d\n", varName, slice, slice, len(slice), cap(slice))
    } else {
        fmt.Println("empty slice, plz alloc")
    }
}

func remove(slice []int, idx int) []int {
    return append(slice[:idx], slice[idx+1:]...)
}

In [63]:
%main
slice := []int{1, 2, 3}
prtSlice("slice org", slice)

// def and alloc by make
var sl1 []int
prtSlice("sl1 def", sl1)
sl1 = make([]int, 18)
prtSlice("sl1 make with len", sl1)

var sl2 = make([]int, 15)
prtSlice("sl2 make with len", sl2)

// make alloc cap and len
sl3 := make([]int, 5, 7)
prtSlice("sl2 make with len and cap", sl3)

// shallow copy
sl4 := sl3
sl41 := sl4[2:4]
sl4[0] = 100
prtSlice("sl3", sl3)
prtSlice("sl4 copy from sl3", sl4)
prtSlice("sl41 slice from sl4", sl41)

// deep copy, limit by dest cap
sl5 := make([]int, 1)
copy(sl5, sl3)
sl5[0] = 200
prtSlice("sl3", sl3)
prtSlice("sl5 deep copy from sl3", sl5)

// append sufficient cap
sl6 := append(sl3, 301)
prtSlice("sl3 append 301(no change, return new slice)", sl3)
prtSlice("sl6 append res", sl6)
// append insufficient cap 
sl7 := append(sl5, 302, 303)
prtSlice("sl5 append 302,303(no change, return new slice)", sl5)
prtSlice("sl5 append res", sl7)

// remove ele by idx
sl3Removed := remove(sl3, 2)
prtSlice("sl3 remove at 2", sl3)
prtSlice("sl3 remove res at 2", sl3Removed)

// multi-dim
multiDim := [][]int{
    {100, 200},
    {300},
    {400, 500},
}
for _, fst := range multiDim {
    for _, value := range fst {
        fmt.Print(value, ",")
    }
}

slice org:	 [1 2 3], type: []int,	 len: 3,	 cap: 3
empty slice, plz alloc
sl1 make with len:	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], type: []int,	 len: 18,	 cap: 18
sl2 make with len:	 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], type: []int,	 len: 15,	 cap: 15
sl2 make with len and cap:	 [0 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl3:	 [100 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl4 copy from sl3:	 [100 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl41 slice from sl4:	 [0 0], type: []int,	 len: 2,	 cap: 5
sl3:	 [100 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl5 deep copy from sl3:	 [200], type: []int,	 len: 1,	 cap: 1
sl3 append 301(no change, return new slice):	 [100 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl6 append res:	 [100 0 0 0 0 301], type: []int,	 len: 6,	 cap: 7
sl5 append 302,303(no change, return new slice):	 [200], type: []int,	 len: 1,	 cap: 1
sl5 append res:	 [200 302 303], type: []int,	 len: 3,	 cap: 3
sl3 remove at 2:	 [100 0 0 0 0], type: []int,	 len: 5,	 cap: 7
sl3 remove res at 2:	 

#### 函数传递注意

slice 在函数传递如果参数不是指针时，传递的是 slice 的 header，此函数参数指向的是原 slice，修改会影响原 slice，但如果使用 append，remove 等会创建新 slice 的操作不会影响原 slice，除非传递指针，map 也类似。
如下例子:

In [65]:
func passSliceHeader(sl []int) {
    // 下面的这些操作都是原 slice
    // 此 sl sl2 指向的不是具体的 slice 而是其 header
    // sl -> slice's header -> slice
    sl2 := sl
    sl2[1] = 200
    prtSlice("sl2 (slice)", sl2)

    // append 会生成新 slice
    // 此 sl 指向的是一个新 slice 的 header，没有改变原 sl 指向的 header 指向的值
    // (sl -> slice's header -> slice) ==> (sl -> new slice's header -> new slice)
    // sl = append(sl, 10) 分解等于
    tmp := append(sl, 10); sl = tmp
    prtSlice("slice new", sl)
}

func passSlicePointer(sp *[]int) {
    // sp 就是原 slice 的 header，直接指向的是具体的 slice 内存空间
    // sl=slice's header -> slice
    sl2 := *sp
    sl2[1] = 200
    prtSlice("sl2 (slice)", sl2)

    // append 会生成新 slice
    // 此 sl 直接指向 slice 内存空间
    // (sl=slice's header -> slice) => (sl=slice's header -> new slice)
    // sl = append(sl, 10) 等于两步
    tmp := append(*sp, 10); *sp = tmp
    prtSlice("slice new", *sp)
}

func main() {
    slice := []int{1, 2, 3}
    prtSlice("slice before pass header", slice)
    passSliceHeader(slice)
    prtSlice("slice after pass header", slice)
    fmt.Println("-----------")

    prtSlice("slice before pass pointer", slice)
    passSlicePointer(&slice)
    prtSlice("slice after pass pointer", slice)
}

slice before pass header:	 [1 2 3], type: []int,	 len: 3,	 cap: 3
sl2 (slice):	 [1 200 3], type: []int,	 len: 3,	 cap: 3
slice new:	 [1 200 3 10], type: []int,	 len: 4,	 cap: 6
slice after pass header:	 [1 200 3], type: []int,	 len: 3,	 cap: 3
-----------
slice before pass pointer:	 [1 200 3], type: []int,	 len: 3,	 cap: 3
sl2 (slice):	 [1 200 3], type: []int,	 len: 3,	 cap: 3
slice new:	 [1 200 3 10], type: []int,	 len: 4,	 cap: 6
slice after pass pointer:	 [1 200 3 10], type: []int,	 len: 4,	 cap: 6


## 5. map

In [98]:
func passMap(mp map[int]int) {
    mp[1] = 100
    mp[100] = 100
    fmt.Printf("mp modified: %v\n", mp)
}

func main() {
    // def map
    var mp1 map[int]int
    if mp1 == nil {
        fmt.Println("nil map")
    }
    // alloc, init with len = 1
    mp1 = make(map[int]int, 1)
    mp1[1] = 10
    mp1[2] = 20
    fmt.Printf("mp1: %v, len=%d\n", mp1, len(mp1))

    // def and init
    mp2 := make(map[int]int, 2)
    mp2[1] = 10
    mp2[2] = 20
    fmt.Printf("mp2: %v\n", mp2)

    // def, init with value
    mp3 := map[int]int {
        1: 10,
        2: 20,
    }
    fmt.Printf("mp3: %v\n", mp3)

    // delete kv by key
    delete(mp3, 1)
    fmt.Printf("mp3 after delete: %v\n", mp3)

    // check key exist
    _, exists := mp3[1]
    if !exists {
        fmt.Println("key=1 not exists!")
    }

    // iter map
    for k, v := range mp3 {
        fmt.Println("key=", k, "value=", v)
    }

    // pass to func, same to slice
    passMap(mp3)
    fmt.Printf("mp3 after pass: %v\n", mp3)
}

nil map
mp1: map[1:10 2:20], len=2
mp2: map[1:10 2:20]
mp3: map[1:10 2:20]
mp3 after delete: map[2:20]
key=1 not exists!
key= 2 value= 20
mp modified: map[1:100 2:20 100:100]
mp3 after pass: map[1:100 2:20 100:100]


## 6. struct

### 6.1 base

与 `array/slice/map` 不同的是 `struct` 声明后就可以直接使用，其默认值不是 `nil`。
> nil is the zero value for `pointers`, `interfaces`, `maps`, `slices`, `channels` and `function` types, representing an uninitialized value.

In [125]:
type Cat struct {
    name string
    age int
}

func passStruct(cat Cat) {
    // copy
    cat.age = 10
}

func passStructPointer(cat *Cat) {
    // (*cat).age simply to cat.age
    // (*cat).age = 10
    cat.age = 10
}

func main() {
    // 声明后就可以直接使用
    var cat Cat
    cat.name = "snow"
    fmt.Printf("cat: %+v\n", cat)
    
    // init with value
    cat2 := Cat{"1", 2}
    fmt.Printf("cat2: %+v\n", cat2)

    // init with new: returned is a pointer to a newly allocated zero value of that type.
    var cat3 Cat
    cat3 = *new(Cat)
    fmt.Printf("cat3: %+v\n", cat3)
    var cat4 *Cat
    cat4 = new(Cat)
    fmt.Printf("cat4: %+v\n", cat4)

    passStruct(cat3)
    fmt.Printf("cat3 after pass: %+v\n", cat3)

    passStructPointer(&cat3)
    fmt.Printf("cat3 after pass pointer: %+v\n", cat3)
}

cat: {name:snow age:0}
cat2: {name:1 age:2}
cat3: {name: age:0}
cat4: &{name: age:0}
cat3 after pass: {name: age:0}
cat3 after pass pointer: {name: age:10}


### 6.2 OOP

1. 封装: `func` 关键字后写类型，可绑定方法到对应类型上，方法名小写则对包外不可见;
2. 关于 interface: 本质是一个指针，只要实现了其方法的 struct 就是其实现类;
3. 组合(可嵌套多个): struct 嵌套了匿名 struct（也可以是指针）或者 interface;
4. 多态: ...

In [191]:
// 封装
type Pet struct {
    nickName string
    age int
}

// 为类定义方法，首字母小写将对外不可见（封装特性）
func (*Pet) Sleep() {
    fmt.Println("pet need a sleep")
}

func (p *Pet) IncAge() {
    p.age += 1
}

func main() {
    pet := Pet{"snow", 1}
    pet.IncAge()
    pet.Sleep()
    fmt.Printf("pet: %+v\n", pet)
}

pet need a sleep
pet: {nickName:snow age:2}


In [192]:
// interface
type Animal interface {
    Sleep()
    Speek() string
    Eat(food string)
}
// 为 Pet 实现 Speek/Eat，则 Pet 就是 Animal 的实现类。
func (p *Pet) Speek() string {
    words := "zzz"
    fmt.Println("speeking...", words)
    return words
}
func (p *Pet) Eat(food string) {
    fmt.Println("eating...", food)
}

func main() {
    var pet Animal
    pet = &Pet{"snow", 1}
    pet.Eat("fish")
}

eating... fish


In [193]:
// 组合结构体
type Dog struct {
    Pet
    color string
}

// 重写父类方法
func (*Dog) Sleep() {
    fmt.Println("dog need a sleep")
}

// 组合结构体指针
type Bunny struct {
    *Pet
    color string
}

// 组合接口
type Bird struct {
    Animal
    color string
}

func main() {
    dog := Dog{Pet{"sniff", 2}, "white"}
    dog.Sleep()
    dog.IncAge()
    fmt.Printf("dog: %+v\n\n", dog)

    var dog2 Dog
    dog2.age = 3
    dog2.color = "brown"
    fmt.Printf("dog2: %+v\n\n", dog2)
    
    bunny := Bunny{}
    bunny.Sleep()
    // bunny.age = 3 pet is nil, cannot assign
    fmt.Printf("bunny: %+v\n\n", bunny)
    
    bunny2 := Bunny{new(Pet), "white"}
    bunny.Sleep()
    bunny2.age = 3
    fmt.Printf("bunny2: %+v\n\n", bunny2)

    bird := Bird{new(Pet), "brown"}
    _ = bird.Speek()
    fmt.Printf("bird: %+v\n", bird)
}

dog need a sleep
dog: {Pet:{nickName:sniff age:3} color:white}

dog2: {Pet:{nickName: age:3} color:brown}

pet need a sleep
bunny: {Pet:<nil> color:}

pet need a sleep
bunny2: {Pet:0xc000096030 color:white}

speeking... zzz
bird: {Animal:0xc000096060 color:brown}


## 7. reflect

一篇很详细的介绍博客: [GO反射的实现原理](https://draveness.me/golang/docs/part2-foundation/ch04-basic/golang-reflect/)

### 7.1 关于类型断言

关于类型断言：
1. 获取变量的动态类型
2. 判断是否实现目标接口, 判断是否实现接口是线性查找是否实现接口的所有函数，如果接口定义的函数越多则速度越慢，所以实际中尽量少用 type-switch 过多类型

In [33]:

type Animal interface {
    Sleep()
    Speek() string
    Eat(food string)
}

type CatNO1 struct {
    name string
}

func (CatNO1) Sleep() {
    fmt.Print("sleep")
}
func (CatNO1) Speek() string {
    fmt.Print("speek")
    return "mow"
}
func (CatNO1) Eat(food string){
    fmt.Println("eat. ", food)
}

func main() {
    a := "str"

    var anyType any
    anyType = a

    fmt.Println(anyType, anyType.(string))

    var ani Animal
    ani = CatNO1{"mow"}
    ani.(CatNO1).Sleep()
    
    var aniAnyType any
    aniAnyType = ani
    
    // type-switch 类型判断
    switch v := aniAnyType.(type) {
    case int:
        fmt.Printf("type=int, value=%v\n", v)
    case string:
        fmt.Printf("type=string, value=%v\n", v)
    case Animal:
        fmt.Printf("type=Animal, value=%+v\n", v)
    default:
        fmt.Printf("unkown type!\n")
    }
}

str str
sleeptype=Animal, value={name:mow}


### 7.1 `TypeOf`/`ValueOf`

- `reflect.TypeOf` 获取类型
- `reflect.ValueOf` 获取数据
